<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/CRYPTO_OPENAI_AGENTKIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://support.kraken.com/articles/360001206766-bitcoin-currency-code-xbt-vs-btc

In [ ]:
!pip install openai-agents ccxt pandas ta -q

In [9]:
# Install required packages (uncomment if running in Colab)
# !pip install openai-agents ccxt pandas ta -q

import asyncio
from agents import Agent, Runner
from agents.tool import function_tool
from pydantic import BaseModel, Field
from IPython.core.getipython import get_ipython
import os
try:
    from google.colab import userdata
except ImportError:
    userdata = None  # Handle non-Colab environments
from openai import OpenAI
import ccxt.async_support as ccxt_async
import pandas as pd
from ta.momentum import RSIIndicator
import time
import logging

# Set up logging for debugging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# --- Test OpenAI API Integration ---
def test_openai_api():
    """Test OpenAI API with a simple prompt."""
    if userdata:
        try:
            api_key = userdata.get('OPENAI_API_KEY')
            if not api_key:
                raise ValueError('OPENAI_API_KEY is not set in Colab secrets.')
        except Exception as e:
            print(f'ERROR: Failed to load OPENAI_API_KEY: {e}')
            print('Please ensure OPENAI_API_KEY is set in Colab Secrets panel.')
            exit(1)
    else:
        api_key = os.getenv('OPENAI_API_KEY')
        if not api_key:
            print('ERROR: OPENAI_API_KEY is not set in environment variables.')
            exit(1)

    client = OpenAI(api_key=api_key)
    try:
        response = client.chat.completions.create(
            model='gpt-4o',
            messages=[
                {'role': 'user', 'content': 'Write a one-sentence bedtime story about a unicorn.'}
            ]
        )
        print(response.choices[0].message.content)
    except Exception as e:
        print(f'ERROR: Failed to call OpenAI API: {e}')

# --- Set Up API Keys ---
def set_api_key():
    """Set OpenAI API key from Colab secrets or environment."""
    if userdata:
        try:
            api_key = userdata.get('OPENAI_API_KEY')
            if not api_key:
                raise ValueError('OPENAI_API_KEY is not set in Colab secrets.')
            os.environ['OPENAI_API_KEY'] = api_key
            print('OpenAI API Key successfully loaded from Colab secrets.')
            print(f'DEBUG: OPENAI_API_KEY in os.environ: {os.getenv("OPENAI_API_KEY")[:4]}...')
        except Exception as e:
            print(f'ERROR: Failed to load OPENAI_API_KEY: {e}')
            print('Please ensure OPENAI_API_KEY is set in Colab Secrets panel.')
            exit(1)
    else:
        if not os.getenv('OPENAI_API_KEY'):
            print('ERROR: OPENAI_API_KEY is not set in environment variables.')
            exit(1)

def set_kraken_keys():
    """Set Kraken API key and secret from Colab secrets or environment."""
    if userdata:
        try:
            kraken_key = userdata.get('KRAKEN')
            kraken_secret = userdata.get('KRAKEN_SECRET')
            if not kraken_key or not kraken_secret:
                raise ValueError('KRAKEN or KRAKEN_SECRET is not set in Colab secrets.')
            print('Kraken credentials successfully loaded from Colab secrets.')
            return kraken_key, kraken_secret
        except Exception as e:
            print(f'ERROR: Failed to load Kraken credentials: {e}')
            print('Please ensure KRAKEN and KRAKEN_SECRET are set in Colab Secrets panel.')
            exit(1)
    else:
        kraken_key = os.getenv('KRAKEN')
        kraken_secret = os.getenv('KRAKEN_SECRET')
        if not kraken_key or not kraken_secret:
            print('ERROR: KRAKEN or KRAKEN_SECRET is not set in environment variables.')
            exit(1)
        return kraken_key, kraken_secret

# --- Pydantic Schemas ---
class AnalysisInput(BaseModel):
    """Structured input for analysis agents."""
    symbol: str = Field(description='Cryptocurrency symbol to analyze (e.g., BTC/USD).')

class TradingSignal(BaseModel):
    """Structured output for trading signals."""
    symbol: str = Field(description='The analyzed symbol.')
    action: str = Field(description='Recommended action: BUY, SELL, or HOLD.')
    confidence: float = Field(description='Confidence score (0.0 to 1.0) in the action.')

# --- Tools for Agents ---
@function_tool
async def get_market_data(symbol: str) -> str:
    """Fetch technical data (RSI, MA, Volume) from Kraken via ccxt, with fallback to simulated data."""
    # Map user-facing symbols to Kraken's API symbols
    symbol_map = {'BTC/USD': 'BTC/USD', 'ETH/USD': 'ETH/USD'}
    ccxt_symbol = symbol_map.get(symbol, None)
    if not ccxt_symbol:
        logger.error(f'Invalid symbol: {symbol}')
        return 'Market data for symbol not found.'

    # Initialize Kraken exchange with ccxt
    try:
        kraken_key, kraken_secret = set_kraken_keys()
        async with ccxt_async.kraken({
            'apiKey': kraken_key,
            'secret': kraken_secret,
            'enableRateLimit': True  # Respect Kraken's rate limits (~1 req/sec)
        }) as exchange:
            logger.info(f'Loading markets for Kraken...')
            await exchange.load_markets()
            available_markets = list(exchange.markets.keys())
            logger.info(f'Available markets: {available_markets[:5]}...')
            if ccxt_symbol not in exchange.markets:
                raise ValueError(f'Kraken does not have market symbol {ccxt_symbol}')

            # Retry logic for API calls
            for attempt in range(3):
                try:
                    logger.info(f'Fetching OHLCV for {ccxt_symbol}, attempt {attempt + 1}')
                    ohlcv = await exchange.fetch_ohlcv(ccxt_symbol, timeframe='1h', limit=200)
                    if not ohlcv:
                        raise ValueError('No OHLCV data returned.')

                    # Create DataFrame
                    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
                    df['close'] = df['close'].astype(float)
                    df['volume'] = df['volume'].astype(float)

                    # Calculate RSI (14-period)
                    rsi = RSIIndicator(df['close'], window=14).rsi().iloc[-1]

                    # Calculate MA_50 (50-period simple moving average)
                    ma_50 = df['close'].rolling(window=50).mean().iloc[-1]

                    # Estimate 24-hour volume change
                    if len(df) >= 48:
                        recent_volume = df['volume'].tail(24).sum()
                        prev_volume = df['volume'].iloc[-48:-24].sum()
                        volume_change = 'high' if recent_volume > prev_volume * 1.2 else 'moderate' if recent_volume > prev_volume * 0.8 else 'low'
                    else:
                        volume_change = 'moderate'

                    return f"{{'RSI': {rsi:.1f}, 'MA_50': {ma_50:.6f}, 'Volume_24h_Change': '{volume_change}'}}"
                except Exception as e:
                    if attempt < 2:
                        logger.warning(f'Attempt {attempt + 1} failed for {symbol}: {e}. Retrying...')
                        await asyncio.sleep(2)
                        continue
                    raise e
    except Exception as e:
        logger.error(f'Failed to initialize Kraken exchange for {symbol}: {e}. Using fallback.')
        # Fallback to simulated data (matched to output)
        if symbol == 'BTC/USD':
            return "{'RSI': 50.0, 'MA_50': 65000, 'Volume_24h_Change': 'moderate'}"
        elif symbol == 'ETH/USD':
            return "{'RSI': 25.0, 'MA_50': 3500.00, 'Volume_24h_Change': 'high'}"
        return 'Market data for symbol not found.'

@function_tool
def check_risk_limits(signal: TradingSignal) -> str:
    """Simulated: Checks if the proposed trade violates portfolio risk limits."""
    print(f'DEBUG: Risk check for {signal.symbol}: action={signal.action}, confidence={signal.confidence}')
    if signal.action == 'BUY' and signal.confidence < 0.7:
        print('DEBUG: Risk check FAILED: Low confidence on BUY signal.')
        return 'RISK CHECK FAILED: Low confidence on BUY signal. Recommend HOLD.'
    if signal.action == 'SELL' and signal.confidence < 0.5:
        print('DEBUG: Risk check FAILED: Low confidence on SELL signal.')
        return 'RISK CHECK FAILED: Low confidence on SELL signal. Recommend HOLD.'
    print('DEBUG: Risk check PASSED.')
    return 'RISK CHECK PASSED: Trade within portfolio limits.'

# --- Define Agents ---
data_agent = Agent(
    name='DataAgent',
    instructions='You are a data expert. Use your tools to provide raw market data without making trading recommendations.',
    tools=[get_market_data],
)

strategy_agent = Agent(
    name='StrategyAgent',
    instructions=(
        'You are an experienced Quant Analyst. '
        'Use the DataAgent tool to get technical data. '
        'Analyze RSI, MA_50, and volume to generate a TradingSignal. '
        'Rules: RSI < 30 → BUY; RSI > 70 → SELL; else HOLD. '
        'Set confidence based on RSI strength (e.g., 0.9 for extreme RSI, 0.5 for neutral).'
    ),
    tools=[
        data_agent.as_tool(
            tool_name='get_data_from_agent',
            tool_description='Use the DataAgent to fetch raw market data for a given cryptocurrency symbol.'
        )
    ],
    output_type=TradingSignal,
)

risk_agent = Agent(
    name='RiskAgent',
    instructions=(
        'You are the final compliance officer. '
        'Receive a TradingSignal and use the check_risk_limits tool. '
        'If the risk check fails, override to TradingSignal with action="HOLD" and confidence=0.0. '
        'Otherwise, output the original TradingSignal.'
    ),
    tools=[check_risk_limits],
    output_type=TradingSignal,
)

triage_agent = Agent(
    name='TriageAgent',
    instructions=(
        'You are the central coordinator for the crypto trading system. '
        'Your job is to: '
        '1. Extract the symbol from the user request (e.g., BTC/USD or ETH/USD). '
        '2. Handoff to StrategyAgent to generate a signal. '
        '3. Handoff the signal to RiskAgent for approval.'
    ),
    handoffs=[strategy_agent, risk_agent],
    output_type=TradingSignal,
)

# --- Run the Workflow ---
async def run_trading_workflow(query: str):
    """Execute the trading workflow for a given query."""
    print(f'--- Running Trading Workflow for Query: "{query}" ---')
    if not os.getenv('OPENAI_API_KEY'):
        print('ERROR: OPENAI_API_KEY is not set. Agents cannot function without a valid API key.')
        return None
    try:
        result = await Runner.run(triage_agent, query)  # Use async run for async tools
        if not result:
            raise ValueError('Agent workflow returned no result.')
    except Exception as e:
        print(f'ERROR: Failed to run workflow: {e}')
        return None
    print('\n[Coordination Complete]')
    final_signal = result.final_output_as(TradingSignal)
    print('-' * 60)
    print(f'FINAL TRADE DECISION for {final_signal.symbol}:')
    print(f'Action: {final_signal.action}')
    print(f'Confidence: {final_signal.confidence:.2f}')
    print('-' * 60)
    return final_signal

# --- Main Execution ---
async def main():
    """Run example queries to demonstrate the trading system."""
    set_api_key()
    print('\nTesting OpenAI API...')
    test_openai_api()
    query_1 = 'Analyze the market for BTC/USD and give a high-confidence signal.'
    await run_trading_workflow(query_1)
    print('\n' + '='*80 + '\n')
    query_2 = 'Analyze a volatile coin like ETH/USD for a quick SELL signal.'
    await run_trading_workflow(query_2)

if __name__ == '__main__':
    if get_ipython() is not None:
        import nest_asyncio
        nest_asyncio.apply()
        asyncio.run(main())
    else:
        asyncio.run(main())

OpenAI API Key successfully loaded from Colab secrets.
DEBUG: OPENAI_API_KEY in os.environ: sk-p...

Testing OpenAI API...
Underneath a blanket of twinkling stars, a gentle unicorn named Luna tiptoed through the moonlit meadow, her shimmering horn casting a cozy glow that lulled every creature into a peaceful slumber.
--- Running Trading Workflow for Query: "Analyze the market for BTC/USD and give a high-confidence signal." ---
Kraken credentials successfully loaded from Colab secrets.

[Coordination Complete]
------------------------------------------------------------
FINAL TRADE DECISION for BTC/USD:
Action: BUY
Confidence: 0.90
------------------------------------------------------------


--- Running Trading Workflow for Query: "Analyze a volatile coin like ETH/USD for a quick SELL signal." ---
Kraken credentials successfully loaded from Colab secrets.

[Coordination Complete]
------------------------------------------------------------
FINAL TRADE DECISION for ETH/USD:
Action: BU